In [ ]:
!pip install pandas plotly dash

In [ ]:

import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df.head()


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                html.Div([html.Label("All Sites"),
                                          dcc.Dropdown(
                                              id = 'site-dropdown',
                                              options = [
                                                  {'label': 'All Sites', 'value': 'ALL'},
                                                  {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                  {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                  {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                  {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                                  value = 'ALL',
                                                  placeholder='Select Sites',
                                                  searchable = True
                                                  )])
                                          ,

             dcc.Graph(id='success-pie-chart'),
             html.Div([
             dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, value=[min_payload, max_payload])

           ]),
dcc.Graph(id='success-payload-scatter-chart')
])


                              # Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class',
        names='Launch Site',
        title='Success count for all launch sites')
        return fig
    else:
      filtered_df2 = spacex_df[spacex_df['Launch Site'] == entered_site]
      filtered_df2 = filtered_df2.groupby(['Launch Site','class']).size().reset_index(name='class count')
      fig = px.pie(filtered_df2, values='class count', names='class', title = 'Total Success Launches for Site {entered_site}')
      return fig



@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatterplot(entered_site, payload_range):
    filtered_df = spacex_df
    if entered_site == 'ALL':
      fig = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version')
      title = 'Correlation between payload mass and success for all sites'
      return fig
    else:
      filtered_df2 = spacex_df[spacex_df['Launch Site'] == entered_site]
      filtered_df2 = filtered_df2.groupby(['Launch Site','class']).size().reset_index(name='class count')
      fig = px.scatter(filtered_df2, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version')
      title = 'Correlation between payload mass and success for Site {entered_site}'
      return fig


if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>